# Deep Nets with TF Abstractions

Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

# The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.

First let's show you how to get the data:

In [1]:
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [8]:
feat_data = wine_data['data']

In [9]:
labels = wine_data['target']

### Train Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
scaled_x_train = scaler.fit_transform(X_train)

In [15]:
scaled_x_test = scaler.transform(X_test)

# Abstractions

## Estimator API

In [16]:
import tensorflow as tf

C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [17]:
from tensorflow import estimator 

In [18]:
X_train.shape

(124, 13)

In [22]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [23]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

W1230 02:52:09.665398 12128 estimator.py:1811] Using temporary folder as model directory: C:\Users\caiyi\AppData\Local\Temp\tmpn1xl1lli


In [24]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [25]:
deep_model.train(input_fn=input_fn,steps=500)

W1230 02:59:11.907624 12128 deprecation.py:323] From C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1230 02:59:11.918595 12128 deprecation.py:323] From C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\inputs\queues\feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1230 02:59:11.920591 12128 deprecation.py:323] From C:\Users\caiyi\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\inputs\queues\feeding_functions.py:500: add_queue_runner (from

In [26]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [28]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

In [29]:
preds 

[{'logits': array([ 1.2348809 ,  0.31200874, -3.1513948 ], dtype=float32),
  'probabilities': array([0.70930886, 0.28186235, 0.00882877], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([ 1.3151093 ,  0.36979663, -3.4575517 ], dtype=float32),
  'probabilities': array([0.7158114 , 0.2781343 , 0.00605422], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([-2.6074202 ,  0.62161446,  1.7640591 ], dtype=float32),
  'probabilities': array([0.00948624, 0.23957735, 0.7509364 ], dtype=float32),
  'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'l

In [30]:
predictions = [p['class_ids'][0] for p in preds]

In [31]:
from sklearn.metrics import confusion_matrix,classification_report

In [32]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.69      0.95      0.80        19
           1       0.88      0.64      0.74        22
           2       1.00      0.92      0.96        13

    accuracy                           0.81        54
   macro avg       0.86      0.84      0.83        54
weighted avg       0.84      0.81      0.81        54



____________
______________

# TensorFlow Keras

### Create the Model

In [728]:
from tensorflow.contrib.keras import models

In [729]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [730]:
from tensorflow.contrib.keras import layers

In [731]:
dnn_keras_model.add(layers.Dense(units=13,input_dim=13,activation='relu'))

In [732]:
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))

In [733]:
dnn_keras_model.add(layers.Dense(units=3,activation='softmax'))

### Compile the Model

In [734]:
from tensorflow.contrib.keras import losses,optimizers,metrics

In [735]:
# explore these
# losses.

In [736]:
#optimizers.

In [744]:
losses.sparse_categorical_crossentropy

<function tensorflow.contrib.keras.python.keras.losses.sparse_categorical_crossentropy>

In [738]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [741]:
dnn_keras_model.fit(scaled_x_train,y_train,epochs=50)

Epoch 1/50
124/124 [==============================] - 0s - loss: 1.1481 - acc: 0.3226     
Epoch 2/50
124/124 [==============================] - 0s - loss: 1.1310 - acc: 0.3226     
Epoch 3/50
124/124 [==============================] - 0s - loss: 1.1141 - acc: 0.3226     
Epoch 4/50
124/124 [==============================] - 0s - loss: 1.1000 - acc: 0.3226     
Epoch 5/50
124/124 [==============================] - 0s - loss: 1.0866 - acc: 0.3226     
Epoch 6/50
124/124 [==============================] - 0s - loss: 1.0746 - acc: 0.3790     
Epoch 7/50
124/124 [==============================] - 0s - loss: 1.0627 - acc: 0.4597     
Epoch 8/50
124/124 [==============================] - 0s - loss: 1.0521 - acc: 0.4758     
Epoch 9/50
124/124 [==============================] - 0s - loss: 1.0422 - acc: 0.5323     
Epoch 10/50
124/124 [==============================] - 0s - loss: 1.0327 - acc: 0.5565     
Epoch 11/50
124/124 [==============================] - 0s - loss: 1.0236 - acc: 0.5968   

In [742]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)

32/54 [================>.............] - ETA: 0s

In [743]:
print(classification_report(predictions,y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       0.91      1.00      0.95        20
          2       1.00      0.87      0.93        15

avg / total       0.97      0.96      0.96        54



# Layers API

https://www.tensorflow.org/tutorials/layers

## Formating Data

In [1]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
wine_data = load_wine()
feat_data = wine_data['data']
labels = wine_data['target']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

In [4]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)
# ONE HOT ENCODED
onehot_y_train = pd.get_dummies(y_train).as_matrix()
one_hot_y_test = pd.get_dummies(y_test).as_matrix()

### Parameters

In [5]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3
learning_rate = 0.01

In [6]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

### Placeholder

In [7]:
X = tf.placeholder(tf.float32,shape=[None,num_feat])
y_true = tf.placeholder(tf.float32,shape=[None,3])

### Activation Function

In [8]:
actf = tf.nn.relu

### Create Layers

In [9]:
hidden1 = fully_connected(X,num_hidden1,activation_fn=actf)

In [10]:
hidden2 = fully_connected(hidden1,num_hidden2,activation_fn=actf)

In [11]:
output = fully_connected(hidden2,num_outputs)

### Loss Function

In [12]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

### Optimizer

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### Init

In [14]:
init = tf.global_variables_initializer()

In [21]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X:scaled_x_train,y_true:y_train})
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

In [25]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(results,y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       1.00      1.00      1.00        22
          2       1.00      1.00      1.00        13

avg / total       1.00      1.00      1.00        54

